# Modeling

## Strategy

Fundamentally, this is an inference project, so the primary goal is to find a reasonably robust model from which we can extract insights as to what should be focused on in Detroit to best improve the HDI.  Nonetheless, we will attempt to fit some less interpretable models to see if we can make more accurate predictions.  Therefore, we will look at regularized multi-variate linear regression, k-nearest neighbors, and decsion tree based models.

In [1]:
# Base libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# sklearn libraries 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNetCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, BaggingRegressor
from sklearn import metrics

# xgboost
from xgboost import XGBRegressor

In [2]:
df = pd.read_csv('./data/clean_data.csv')
hdi = pd.read_csv('./data/country_hdi.csv')

Here we'll make a train and test set using all of the numeric data in our dataframe with the exception of the HDI (which is our target) and the calculated intermediate indices that were used to calculate the HDI.  The data for Wayne County, MI will also be seperated from the whole to use as a final check at the end of modeling.

In [3]:
county = 'Wayne County'
state = 'MI'
county_input = county + ', ' + state
county_x = df[df['Description'] == county_input].select_dtypes(include=[np.number]).drop(columns = ['LE', 'II', 'MYS', 'EI', 'HDI'])
county_y = df[df['Description'] == county_input]['HDI']

X = df[df['Description'] != county_input].select_dtypes(include=[np.number]).drop(columns = ['LE', 'II', 'MYS', 'EI', 'HDI'])
y = df[df['Description'] != county_input]['HDI']

First we will remove highly colinear features.  The strategy here is to compare every pair of features to see if they are colinear and, if they are, remove the one that is less correlated to the target feature of HDI.  This will be important in the final interpretation because if, for example, health is the most important feature and income is the second most important we would like to see both of these results instead of simply a series of highly correlated health features.

We experimented with a number of different colinearity coefficients to optimize the results, ultimately settling on aprroximately two-thirds (0.67) as the best trade off between predictive power of the model and usefulness of results.

A list of the highly colinear features that are removed from the data is returned from this function.

In [4]:
def colinearity_remover(df, max_corr, list_of_keepers = []):
    corr = df.corr()
    x, y = corr.shape
    drop_list = []
    target = df.columns[-1]    
    
    for i in range(x-1):
        for j in range (y-1):
            if abs(corr.iloc[i, j]) > max_corr and i != j:
                if corr[target][i] > corr[target][j]:
                    drop_list.append(corr[target].index[j])
                else:
                    drop_list.append(corr[target].index[i])
                                 
    drop_list = list(set(drop_list))
    
    for keeper in list_of_keepers:
        drop_list.remove(keeper)
    
    
    
    return drop_list

drop_list = colinearity_remover(X, 0.67, list_of_keepers = [])
drop_list

['emp. in social assistance and outreach industries per 10,000 population',
 'Violent Crime Rate',
 'Interest on debt per pupil',
 'Worked Outside County of Residence   rate',
 'GDP per worker growth, 2001-2016',
 'Percent of the civilian population that is uninsured',
 'Total workers 16+ commuting to work, 2016 rate',
 'Percent of returns with self-emp. retirement plans',
 'Percent of current spending for instruction',
 'Percent of working populaiton that drove to work alone',
 'Violent crime events per 1,000 population',
 'Poor physical health days per month',
 'Percent of working population that walked to work',
 'Proportion of single-parent households',
 'Poor mental health days per month',
 'Percent of returns with contributions',
 '% Children in Poverty',
 'Percent of returns that are farm returns',
 'Percent of working population that took public transit (excluding taxicabs) to work',
 'Percent of total revenue from local sources',
 '2010 population',
 'Teen Birth Rate',
 'Perce

In [5]:
print(f"The shape of the data before colinearity removal is: {X.shape}")
X.drop(columns = drop_list, inplace = True)
county_x.drop(columns = drop_list, inplace = True)
print(f"After removal: {X.shape}")

The shape of the data before colinearity removal is: (2968, 186)
After removal: (2968, 117)


The data are split into train and test sets and scaled as our features have vastly different orders of magnitude.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 48228)
ss = StandardScaler()
Z_train = ss.fit_transform(X_train)
Z_test = ss.transform(X_test)
county_z = ss.transform(county_x)

First a multi-variate linear model is fitted with optimal tunign parameters determined with a grid search.

In [7]:
alphas = np.linspace(0.0005, 0.0015, 100)
enet_ratios = np.linspace(0.21, 0.22, 100)

enet = ElasticNetCV(alphas = alphas, l1_ratio=enet_ratios, cv = 5, max_iter = 3000, n_jobs = -1)

In [8]:
enet.fit(Z_train, y_train)
enet.alpha_

0.0009545454545454546

In [9]:
enet.l1_ratio_

0.2197979797979798

Here we inspect the most important features of the linear model.  Note that these features have the most impact overall, but will not necessarily be the most important features for Wayne County to focus on.  If, for example, Wayne County happens to not have a large porportion of its adults in the "working" age range of 18 to 64, this feature will not be important as a goal for Wayne County to focus on.

In [10]:
enet_coefs = pd.DataFrame([X_train.columns, enet.coef_], index = ['parameter', 'coef']).T
enet_coefs['importance'] = enet_coefs['coef'].map(lambda x: abs(x))
enet_coefs.sort_values(by = 'importance', ascending = False).head(10)

,parameter,coef,importance
19,Percent of population that didn't work over th...,-0.008419,0.008419
21,Percent of returns with itemized deductions,0.00585,0.005850
100,% Smokers,-0.003856,0.003856
64,Total civilian population aged 18-64 rate,0.0035,0.003500
23,Percent of returns with salaries and wages,-0.0028,0.002800
32,"Population growth, 2010-2016",0.002669,0.002669
17,Percent of civilian population aged 25-34 that...,-0.00264,0.002640
41,Rural-urban continuum code (1-9),-0.002192,0.002192
13,"Non-rent seeking organizations per 10,000 popu...",0.002075,0.002075
43,Share of emp. in all local industries,0.001935,0.001935


Next we look at a K-nearest neighbors regressor.

In [11]:
knr_params = {
    'n_neighbors':range(8,12,1),
}

knr = GridSearchCV(
    KNeighborsRegressor(),
    knr_params,
    cv=5,
    n_jobs = -1,
    verbose = 2
)

knr.fit(Z_train, y_train)
knr.best_params_

Fitting 5 folds for each of 4 candidates, totalling 20 fits


{'n_neighbors': 11}

Then a random forest model.

In [12]:
rfr_params = {
    'n_estimators': [93, 95, 97, 99, 101],
    'max_depth': [14, 15, 16, 17, 18],
    'max_features': ['auto']
}

rfr = GridSearchCV(
    RandomForestRegressor(),
    rfr_params,
    cv=5,
    n_jobs = -1,
    verbose = 2
)

rfr.fit(Z_train, y_train)
rfr.best_params_

Fitting 5 folds for each of 25 candidates, totalling 125 fits


{'max_depth': 17, 'max_features': 'auto', 'n_estimators': 99}

Finally, an XG boost regressor.

In [13]:
xg_params = {
    'n_estimators': [97, 99, 101, 103],
    'max_depth': [2, 3, 4],
    'reg_alpha': np.linspace(0.07, 0.08, 10)
}

xg = GridSearchCV(
    XGBRegressor(),
    xg_params,
    cv=5,
    n_jobs = -1,
    verbose = 2
)

xg.fit(Z_train, y_train)
xg.best_params_

Fitting 5 folds for each of 120 candidates, totalling 600 fits


{'max_depth': 3, 'n_estimators': 101, 'reg_alpha': 0.07555555555555556}

The results are summarized below.

In [14]:
for name, model in {'Elastic Net': enet, 'K-Nearest': knr, 'Random Forest': rfr, 'XG Boost': xg}.items():
    pred = model.predict(Z_test)
    r2 = round(metrics.r2_score(pred, y_test), 3)
    rmse = round(metrics.mean_squared_error(pred, y_test, squared = False), 3)   
    
    print(f"The R-squared score for the {name} model is: {r2}")
    print(f"The root mean squared error for the {name} model is: {rmse} \n")

The R-squared score for the Elastic Net model is: 0.744
The root mean squared error for the Elastic Net model is: 0.015 

The R-squared score for the K-Nearest model is: 0.507
The root mean squared error for the K-Nearest model is: 0.016 

The R-squared score for the Random Forest model is: 0.787
The root mean squared error for the Random Forest model is: 0.013 

The R-squared score for the XG Boost model is: 0.832
The root mean squared error for the XG Boost model is: 0.012 



Note that while the XG Boost model has the best R2 score and root mean square error, we will not carry it forward as the XG Boost model is fairly uninterpretable.  Instead our final report will be constructed from the Elastic Net linear regression.  If predicting HDI as accurately as possible were the most desired goal we would use the XG Boost model.  However, for interpretability the Elastic Net model is vastly preferable despite it's poorer predictive performance.

## Report

The report below shows the top 5 areas where Wayne County both underperformes the national average **and** has the most opportunity to improve.  Note that according to our previous results ```Percent of returns with itemized deductions``` was the second most important feature in predicting HDI; however, it does not appear in the top portion of the report.  This is because Wayne County already performs well in this particular category and does not need to "work" on this issue.  Instead, a feature like ```% Low birthweight```, which was only the 20th most important feature, appears as the number 4 priority.  This is because Wayne County is significantly below the national average in this parameter and could, therefore, see the largest positive impact on its HDI by focusing on this issue.  


In [15]:
report = pd.DataFrame(X.mean(), columns = ['Mean Value'])
report['County Value'] = county_x.T
Z = np.concatenate((Z_train, Z_test), axis = 0)
report['County SDs from Mean'] = (county_z - Z.mean(axis = 0)).T
report['Amount of Change'] = (ss.inverse_transform(county_z) - ss.inverse_transform(Z.mean(axis = 0))).T
report['Impact on HDI'] = ((Z.mean(axis = 0) - county_z) * enet.coef_).T
for feature in report.columns:
    report[feature] = report[feature].apply(lambda x: round(x,3))
report.sort_values(by = 'Impact on HDI', ascending = False)

,Mean Value,County Value,County SDs from Mean,Amount of Change,Impact on HDI
Motor Vehicle Theft rate per 100000,106.465,859.079,7.354,752.615,0.006
Percent of population that didn't work over the past year,26.435,32.402,0.641,5.966,0.005
% Smokers,21.345,24.000,0.693,2.655,0.003
% Low birthweight,8.134,11.000,1.580,2.866,0.002
"Population growth, 2010-2016",0.343,-2.911,-0.696,-3.253,0.002
...,...,...,...,...,...
% With Access to Exercise Opportunities,64.255,95.000,1.490,30.745,-0.002
Percent of returns with itemized deductions,21.344,24.243,0.348,2.899,-0.002
Rural-urban continuum code (1-9),4.359,1.000,-1.517,-3.359,-0.003
Total civilian population aged 18-64 rate,0.578,0.617,0.877,0.039,-0.003


### International Comparison

Below we compare Wayne County with the international community as a whole to provide a bit of context.  Note that this analysis needs to be viewed carefully as HDI variability within a nation is often of larger magnitude than the variabilities in the *average* HDI between nations.

In [16]:
def get_hdi(my_hdi):
    compare = 1
    for i in range(len(hdi)):
        this_hdi = hdi.iloc[i, 1]
        if abs(my_hdi - this_hdi) < compare:
            compare = abs(my_hdi - this_hdi)

            country = hdi.iloc[i, 0]
        
    return country

In [17]:
county_hdi = float(df[df['Description'] == county_input]['HDI'])
improvement = report.sort_values(by = 'Impact on HDI', ascending = False)['Impact on HDI'][0:4].sum()
currently_like = get_hdi(county_hdi) 
would_be_like = get_hdi(county_hdi + improvement)
print(f"With an HDI of {round(county_hdi, 3)}, {county_input}, compared to the international community, currently has an HDI most similar to {currently_like}.")
print(f"According to our model, if {county_input} successfully remediated the top 5 items it could have an HDI more simalar to {would_be_like}.")  

With an HDI of 0.841, Wayne County, MI, compared to the international community, currently has an HDI most similar to  Brunei.
According to our model, if Wayne County, MI successfully remediated the top 5 items it could have an HDI more simalar to  Slovakia.


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNetCV

df = pd.read_csv('./data/clean_data.csv')
hdi = pd.read_csv('./data/country_hdi.csv')

county = input('Input County')
state = input('Input Two Letter State Code')
county_input = county + ', ' + state
county_x = df[df['Description'] == county_input].select_dtypes(include=[np.number]).drop(columns = ['LE', 'II', 'MYS', 'EI', 'HDI'])
county_y = df[df['Description'] == county_input]['HDI']

X = df[df['Description'] != county_input].select_dtypes(include=[np.number]).drop(columns = ['LE', 'II', 'MYS', 'EI', 'HDI'])
y = df[df['Description'] != county_input]['HDI']


def colinearity_remover(df, max_corr, list_of_keepers = []):
    corr = df.corr()
    x, y = corr.shape
    drop_list = []
    target = df.columns[-1]    
    
    for i in range(x-1):
        for j in range (y-1):
            if abs(corr.iloc[i, j]) > max_corr and i != j:
                if corr[target][i] > corr[target][j]:
                    drop_list.append(corr[target].index[j])
                else:
                    drop_list.append(corr[target].index[i])
                                 
    drop_list = list(set(drop_list))
    
    for keeper in list_of_keepers:
        drop_list.remove(keeper)
    
    
    
    return drop_list

drop_list = colinearity_remover(X, 0.67, list_of_keepers = [])
X.drop(columns = drop_list, inplace = True)
county_x.drop(columns = drop_list, inplace = True)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 48228)
ss = StandardScaler()
Z_train = ss.fit_transform(X_train)
Z_test = ss.transform(X_test)
county_z = ss.transform(county_x)


alphas = [0.0009545454545454546]
enet_ratios = [0.2197979797979798]

enet = ElasticNetCV(alphas = alphas, l1_ratio=enet_ratios, cv = 5, max_iter = 3000, n_jobs = -1)

enet.fit(Z_train, y_train)

report = pd.DataFrame(X.mean(), columns = ['Mean Value'])
report['County Value'] = county_x.T
Z = np.concatenate((Z_train, Z_test), axis = 0)
report['County SDs from Mean'] = (county_z - Z.mean(axis = 0)).T
report['Amount of Change'] = (ss.inverse_transform(county_z) - ss.inverse_transform(Z.mean(axis = 0))).T
report['Impact on HDI'] = ((Z.mean(axis = 0) - county_z) * enet.coef_).T
for feature in report.columns:
    report[feature] = report[feature].apply(lambda x: round(x,3))


def get_hdi(my_hdi):
    compare = 1
    for i in range(len(hdi)):
        this_hdi = hdi.iloc[i, 1]
        if abs(my_hdi - this_hdi) < compare:
            compare = abs(my_hdi - this_hdi)

            country = hdi.iloc[i, 0]
        
    return country
    
county_hdi = float(df[df['Description'] == county_input]['HDI'])
improvement = report.sort_values(by = 'Impact on HDI', ascending = False)['Impact on HDI'][0:4].sum()
currently_like = get_hdi(county_hdi) 
would_be_like = get_hdi(county_hdi + improvement)
print(f"With an HDI of {round(county_hdi, 3)}, {county_input}, compared to the international community, currently has an HDI most similar to {currently_like}.")
print(f"According to our model, if {county_input} successfully remediated the top 5 items it could have an HDI more simalar to {would_be_like}.")    
    
report.sort_values(by = 'Impact on HDI', ascending = False)

Input County Cook County
Input Two Letter State Code IL


With an HDI of 0.894, Cook County, IL, compared to the international community, currently has an HDI most similar to  Malta.
According to our model, if Cook County, IL successfully remediated the top 5 items it could have an HDI more simalar to  France.


,Mean Value,County Value,County SDs from Mean,Amount of Change,Impact on HDI
Percent of returns with salaries and wages,80.975,83.045,0.448,2.070,0.002
Motor Vehicle Theft rate per 100000,106.624,386.303,2.614,279.679,0.002
"Non-rent seeking organizations per 10,000 population",12.449,5.542,-1.071,-6.907,0.002
Standardized score of January mean sunlight,0.003,-0.648,-0.649,-0.651,0.001
% Low birthweight,8.135,9.000,0.476,0.865,0.001
...,...,...,...,...,...
% Smokers,21.348,16.000,-1.387,-5.348,-0.004
Share of emp. in top 2 local industries,0.175,0.298,2.136,0.123,-0.004
Total civilian population aged 18-64 rate,0.578,0.639,1.400,0.061,-0.005
Civilian population aged 25 and up,66726.335,3546688.000,15.677,3479961.665,-0.006


### Conclusion

While the HDI of a region is a somewhat reductionist metric, this model does have some advantages in identifying some key metrics where a region is both lagging behind the national average ***and*** where improvement in this metric is likely to have the highest impact on the HDI.  This should not be construed as a definitive way to set priorities, but it is helpful in knowing which areas are worthy of additional research.  So, using the report for Wayne County above, if one wanted to launch an anti-smoking campaign, that does seem likely to be a worthwhile endevour.  Conversely, we see that Wayne County is not lacking in access to exercise facilities so a program to provide gym time to low income residents may not be the best use of time and resources.